In [0]:
import pyspark
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName('LR_project').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
data=spark.read.csv('/FileStore/tables/cruise_ship_info.csv',inferSchema=True, header=True)

In [0]:
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [0]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 20 rows

In [0]:
for ship in data.head(5):
  print(ship)
  print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)

In [0]:
# Since cruise _line come up with same for many ships, we have to calculate their number inorder to predict the output.
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer=StringIndexer(inputCol='Cruise_line',outputCol='Cruise_category')
indexed=indexer.fit(data)
final_indexed=indexed.transform(data)

In [0]:
final_indexed.head(3)
# the cruise_line is categorised

Out[70]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_category=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_category=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_category=1.0)]

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
final_indexed.columns

Out[71]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_category']

In [0]:
assembler=VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','Cruise_category'],
                         outputCol='features')

In [0]:
output=assembler.transform(final_indexed)

In [0]:
output.printSchema()
#feature column is now added to the schema 

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- Cruise_category: double (nullable = false)
-- features: vector (nullable = true)

In [0]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+--------------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_category| features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+--------------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|[6.0,30.276999999...|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|[6.0,30.276999999...|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|[26.0,47.262,14.8...|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|[11.0,110.0,29.74...|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|[17.0,101.353,26....|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[22.0,70.367,20.5...|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[15.0,70.367,20.5...|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|[23.0,70.367,20.5...|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[19.0,70.367,20.5...|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|[6.0,110.23899999...|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|[10.0,110.0,29.74...|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|[28.0,46.052,14.5...|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[18.0,70.367,20.5...|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[17.0,70.367,20.5...|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|[11.0,86.0,21.24,...|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|[8.0,110.0,29.74,...|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|[9.0,88.5,21.24,9...|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[15.0,70.367,20.5...|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|[12.0,88.5,21.24,...|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[20.0,70.367,20.5...|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+--------------------+
only showing top 20 rows

In [0]:
output.select('features').show()

+--------------------+
 features|
+--------------------+
[6.0,30.276999999...|
[6.0,30.276999999...|
[26.0,47.262,14.8...|
[11.0,110.0,29.74...|
[17.0,101.353,26....|
[22.0,70.367,20.5...|
[15.0,70.367,20.5...|
[23.0,70.367,20.5...|
[19.0,70.367,20.5...|
[6.0,110.23899999...|
[10.0,110.0,29.74...|
[28.0,46.052,14.5...|
[18.0,70.367,20.5...|
[17.0,70.367,20.5...|
[11.0,86.0,21.24,...|
[8.0,110.0,29.74,...|
[9.0,88.5,21.24,9...|
[15.0,70.367,20.5...|
[12.0,88.5,21.24,...|
[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows

In [0]:
final_data=output.select('features','crew')

In [0]:
final_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [0]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 106|
 mean|7.500000000000011|
 stddev|3.271201614086178|
 min| 0.59|
 max| 13.6|
+-------+-----------------+

In [0]:
test_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 52|
 mean|8.393846153846154|
 stddev| 3.89980198282926|
 min| 0.6|
 max| 21.0|
+-------+-----------------+

In [0]:
ship_lr=LinearRegression(labelCol='crew')

In [0]:
trained_ship_model=lr.fit(train_data)

In [0]:
cruise_results=trained_ship_model.evaluate(test_data)

In [0]:
cruise_results.residuals.show()

+--------------------+
 residuals|
+--------------------+
 1.0657719193021222|
 -0.669323600454887|
 0.7053237412158868|
 0.5127472128338226|
 -0.1442763624345904|
 -1.140484641503468|
-0.07429722431947461|
 1.0051824638576967|
 -0.6596193307089031|
 0.859066419929313|
 1.3836253864926746|
 1.3836253864926746|
 0.19828516294691845|
 -0.2822896915028945|
 -0.486481551717441|
-0.24322618661693873|
 0.17711770852675635|
 -0.7842049734867054|
 0.9835978897221942|
 7.317665147915109|
+--------------------+
only showing top 20 rows

In [0]:
cruise_results.rootMeanSquaredError

Out[90]: 1.2971666681463165

In [0]:
cruise_results.r2

Out[92]: 0.8871920733637618

In [0]:
cruise_results.meanAbsoluteError

Out[100]: 0.8261650297600374

In [0]:
cruise_results.meanSquaredError

Out[101]: 1.6826413649498162

In [0]:
unlabeled_data=test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[4.0,220.0,54.0,1...|
[6.0,30.276999999...|
[6.0,93.0,23.94,9...|
[6.0,113.0,37.82,...|
[6.0,158.0,43.7,1...|
[7.0,89.6,25.5,9....|
[7.0,158.0,43.7,1...|
[8.0,91.0,22.44,9...|
[9.0,85.0,19.68,9...|
[9.0,88.5,21.24,9...|
[9.0,113.0,26.74,...|
[9.0,113.0,26.74,...|
[9.0,116.0,26.0,9...|
[10.0,58.825,15.6...|
[10.0,86.0,21.14,...|
[10.0,105.0,27.2,...|
[11.0,58.6,15.66,...|
[11.0,85.0,18.48,...|
[11.0,90.0,22.4,9...|
[11.0,110.0,29.74...|
+--------------------+
only showing top 20 rows

In [0]:
predictions=trained_ship_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[4.0,220.0,54.0,1...|19.934228080697878|
[6.0,30.276999999...| 4.219323600454887|
[6.0,93.0,23.94,9...|10.384676258784113|
[6.0,113.0,37.82,...|11.487252787166177|
[6.0,158.0,43.7,1...| 13.74427636243459|
[7.0,89.6,25.5,9....|11.010484641503467|
[7.0,158.0,43.7,1...|13.674297224319474|
[8.0,91.0,22.44,9...| 9.994817536142303|
[9.0,85.0,19.68,9...| 9.349619330708903|
[9.0,88.5,21.24,9...| 9.440933580070688|
[9.0,113.0,26.74,...|10.996374613507326|
[9.0,113.0,26.74,...|10.996374613507326|
[9.0,116.0,26.0,9...|10.801714837053082|
[10.0,58.825,15.6...|7.2822896915028945|
[10.0,86.0,21.14,...| 9.68648155171744|
[10.0,105.0,27.2,...|10.923226186616938|
[11.0,58.6,15.66,...| 7.422882291473243|
[11.0,85.0,18.48,...| 8.784204973486705|
[11.0,90.0,22.4,9...|10.016402110277806|
[11.0,110.0,29.74...|11.782334852084892|
+--------------------+------------------+
only showing top 20 rows

In [0]:
final_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [0]:
# to find the correlation between two columns, (eg: crew - cabin, crew - passengers), we use pearson correlation

In [0]:
from pyspark.sql.functions import corr

In [0]:
data. select(corr('crew','passengers')).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [0]:
data.select(corr('crew','Cabins')).show()

+------------------+
corr(crew, Cabins)|
+------------------+
0.9508226063578497|
+------------------+